<a href="https://colab.research.google.com/github/AlexKressner/WS24_Supply_Chain_Optimierung/blob/main/NetworkDesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network Design

In [264]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [265]:
! pip install -q pyscipopt

In [266]:
import pandas as pd
from pyscipopt import Model, quicksum

## Daten aus csv-Datei laden



In [267]:
! ls WS24_Supply_Chain_Optimierung/Daten/NetworkDesign

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [268]:
folder = "WS24_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [269]:
path = f"{folder}/NetworkDesign_Invest_Kapa.csv"

In [270]:
kapa=pd.read_csv(path, sep=";")

In [271]:
kapa.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [272]:
path = f"{folder}/NetworkDesign_Produktionskosten.csv"

In [273]:
produktionskosten=pd.read_csv(path, sep=";")

In [274]:
produktionskosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [275]:
path = f"{folder}/NetworkDesign_Nachfrage.csv"

In [276]:
nachfrage=pd.read_csv(path, sep=";")

In [277]:
nachfrage.head()

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


## Optimierungsmodell

In [278]:
scip = Model()

### Indexmengen

In [279]:
I = produktionskosten["Produktionsstandort"].unique().tolist() # Menge der Produktionsstandorte

In [280]:
I

['Charleston', 'Curitiba', 'Hamburg', 'Quanzhou', 'East London']

In [281]:
J = nachfrage["Markt"].tolist() #Menge der Märkte

In [282]:
J.append("Arabische_Halbinsel")

In [283]:
J

['USA', 'Südamerika', 'Europa', 'Asien', 'Afrika', 'Arabische_Halbinsel']

In [284]:
A = kapa["Ausbaustufe"].unique().tolist() # Menge der Ausbaustufen an einem Standort

In [285]:
A

['klein', 'groß']

### Entscheidungsvariablen

In [286]:
# Definition der Entscheidungsvariablen
# X sind die Flussvariablen, d.h. Herstell- und Transportmenge von i nach j
X={}
for i in I:
  for j in J:
    X[i,j] = scip.addVar(vtype="C", name=f"{i},{j}")

In [287]:
# Definition der Entscheidungsvariablen
# Y sind die Strukturvariablen, d.h. ob und mit welchen Kapazitäten ein Standort ausgebaut wird
Y={}
for i in I:
  for a in A:
    Y[i,a] = scip.addVar(vtype="B", name=f"{i},{a}")

In [288]:
print('Anzahl Entscheidungsvariablen =', len(scip.getVars()))

Anzahl Entscheidungsvariablen = 40


### Parameter

In [289]:
i = "Produktionsstandort"
j = "Markt"

In [290]:
# Herstell- und Transportkosten
produktionskosten.set_index([i,j], inplace=True)

In [291]:
cp = produktionskosten.to_dict("dict")["Produktionskosten"]

In [292]:
cp

{('Charleston', 'USA'): 81,
 ('Charleston', 'Südamerika'): 92,
 ('Charleston', 'Europa'): 101,
 ('Charleston', 'Asien'): 130,
 ('Charleston', 'Afrika'): 115,
 ('Curitiba', 'USA'): 117,
 ('Curitiba', 'Südamerika'): 77,
 ('Curitiba', 'Europa'): 108,
 ('Curitiba', 'Asien'): 98,
 ('Curitiba', 'Afrika'): 100,
 ('Hamburg', 'USA'): 102,
 ('Hamburg', 'Südamerika'): 105,
 ('Hamburg', 'Europa'): 95,
 ('Hamburg', 'Asien'): 119,
 ('Hamburg', 'Afrika'): 111,
 ('Quanzhou', 'USA'): 115,
 ('Quanzhou', 'Südamerika'): 125,
 ('Quanzhou', 'Europa'): 90,
 ('Quanzhou', 'Asien'): 59,
 ('Quanzhou', 'Afrika'): 74,
 ('East London', 'USA'): 142,
 ('East London', 'Südamerika'): 100,
 ('East London', 'Europa'): 103,
 ('East London', 'Asien'): 105,
 ('East London', 'Afrika'): 71}

In [293]:
cp['Charleston','Arabische_Halbinsel']=125
cp['Curitiba','Arabische_Halbinsel']=100
cp['Hamburg','Arabische_Halbinsel']=100
cp['Quanzhou','Arabische_Halbinsel']=75
cp['East London','Arabische_Halbinsel']=85

In [294]:
nachfrage

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


In [295]:
# Nachfrage aus den Märtken
nachfrage.set_index(["Markt"], inplace=True)

In [296]:
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [297]:
d["Arabische_Halbinsel"] = 5

In [298]:
d

{'USA': 12,
 'Südamerika': 8,
 'Europa': 14,
 'Asien': 16,
 'Afrika': 7,
 'Arabische_Halbinsel': 5}

In [299]:
# Kapazitäten der Ausbaustufen und Investitionskosten
kapazitaeten = kapa.set_index("Ausbaustufe")

In [300]:
cap = kapazitaeten.to_dict("dict")["Kapazitäten"]

In [301]:
cap

{'klein': 10, 'groß': 20}

In [302]:
cf = kapa.set_index(["Produktionsstandort","Ausbaustufe"]).to_dict("dict")["Investitionen"]

In [303]:
cf

{('Charleston', 'klein'): 6000,
 ('Charleston', 'groß'): 9000,
 ('Curitiba', 'klein'): 4500,
 ('Curitiba', 'groß'): 6750,
 ('Hamburg', 'klein'): 6500,
 ('Hamburg', 'groß'): 9750,
 ('Quanzhou', 'klein'): 4100,
 ('Quanzhou', 'groß'): 6150,
 ('East London', 'klein'): 4000,
 ('East London', 'groß'): 6000}

**Variante 1:** Subventionen

In [304]:
cf['Charleston','klein'] = cf['Charleston','klein']*0.8

In [305]:
cf['Charleston','groß'] = cf['Charleston','groß']*0.8

### Zielfunktion

min $K=\sum_{i,j} cp_{ij}* X_{ij} + \sum_{i,a} cf_{ia}* Y_{ia}$

In [306]:
# Maximierung der gesamten Deckungsbeiträge
scip.setObjective(quicksum(cp[i,j]*X[i,j] for i in I for j in J) + quicksum(cf[i,a]*Y[i,a] for i in I for a in A), sense="minimize")

### Nebenbedingungen

$\sum_i X_{ij} \ge d_j, ∀j$  

In [307]:
# Nachfrage in den Märkten erfüllen
for j in J:
  scip.addCons(quicksum(X[i,j] for i in I)>=d[j])

$\sum_j X_{ij} \le \sum_a cap_{a} * Y_{ia}, ∀i$

In [308]:
# Kapazitäten aufbauen und einhalten
for i in I:
  scip.addCons(quicksum(X[i,j] for j in J)<= sum(cap[a]*Y[i,a] for a in A))

## Optimierung

In [309]:
scip.optimize()
print(scip.getStatus())

optimal


In [310]:
print('LÖSUNG:')
print('Zielfunktionswert (Kosten) =', scip.getObjVal())
print("\n")
print("PRODUKTIONS-MARKT-ZUTEILUNG")
for j in J:
  print(f"{j}:")
  for i in I:
    if scip.getVal(X[i,j])>0:
      print('\t', f'{i,j} =', round(scip.getVal(X[i,j])))
print("\n")
print("INVESTITIONEN")
for i in I:
  for a in A:
    if scip.getVal(Y[i,a])>0:
      print(f'{i,a} =', round(scip.getVal(Y[i,a])))

LÖSUNG:
Zielfunktionswert (Kosten) = 28146.0


PRODUKTIONS-MARKT-ZUTEILUNG
USA:
	 ('Curitiba', 'USA') = 12
	 ('Quanzhou', 'USA') = 0
Südamerika:
	 ('Curitiba', 'Südamerika') = 8
Europa:
	 ('Quanzhou', 'Europa') = 14
	 ('East London', 'Europa') = 0
Asien:
	 ('Quanzhou', 'Asien') = 16
Afrika:
	 ('East London', 'Afrika') = 7
Arabische_Halbinsel:
	 ('East London', 'Arabische_Halbinsel') = 5


INVESTITIONEN
('Curitiba', 'groß') = 1
('Quanzhou', 'klein') = 1
('Quanzhou', 'groß') = 1
('East London', 'groß') = 1
